# VacationPy
----


In [61]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [62]:
# Load csv file created from WeatherPy notebook
weather_csv_file = "./output_data/weatherPy.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,roald,100.0,NO,1.660448e+09,94.0,62.5833,6.1333,61.16,8.05
1,1,palasa,100.0,IN,1.660448e+09,92.0,18.7667,84.4167,77.72,13.47
2,2,saldanha,28.0,ZA,1.660448e+09,70.0,-33.0117,17.9442,50.05,18.77
3,3,hobart,75.0,AU,1.660448e+09,88.0,-42.8794,147.3294,58.86,12.66
4,4,kodiak,100.0,US,1.660448e+09,88.0,57.7900,-152.4072,54.91,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [63]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 6]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
72,80,russell,0.0,US,1.660448e+09,94.0,32.3502,-85.1999,74.43,0.00
92,100,carnarvon,0.0,AU,1.660448e+09,55.0,-24.8667,113.6333,70.52,1.66
166,183,havelock,0.0,US,1.660448e+09,78.0,34.8791,-76.9013,75.94,5.75
186,203,sorrento,0.0,IT,1.660448e+09,90.0,40.6260,14.3744,72.97,0.00
296,327,mayo,0.0,US,1.660448e+09,82.0,38.8876,-76.5119,72.99,0.00
444,493,fairbanks,0.0,US,1.660448e+09,63.0,64.8378,-147.7164,73.60,0.00
518,571,vernon,0.0,CA,1.660448e+09,42.0,50.2581,-119.2691,74.01,1.14
550,604,andros,0.0,GR,1.660448e+09,54.0,37.8333,24.9333,73.96,4.00
561,616,rockhampton,0.0,AU,1.660448e+09,41.0,-23.3833,150.5000,76.89,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 72: russell.
Closest hotel is Macs.
------------
Retrieving Results for Index 92: carnarvon.
Closest hotel is Hospitality Carnarvon.
------------
Retrieving Results for Index 166: havelock.
Closest hotel is Sherwood Motel.
------------
Retrieving Results for Index 186: sorrento.
Closest hotel is Hilton Sorrento Palace.
------------
Retrieving Results for Index 296: mayo.
Closest hotel is Resorts Resource Group International.
------------
Retrieving Results for Index 444: fairbanks.
Closest hotel is Westmark Fairbanks Hotel & Conference Center.
------------
Retrieving Results for Index 518: vernon.
Closest hotel is Prestige Vernon Lodge and Conference Centre.
------------
Retrieving Results for Index 550: andros.
Closest hotel is Micra Anglia Hotel - Andros.
------------
Retrieving Results for Index 561: rockhampton.
Closest hotel is Mercure Rockhampton.
------------


In [66]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
72,80,russell,0.0,US,1.660448e+09,94.0,32.3502,-85.1999,74.43,0.00,Macs
92,100,carnarvon,0.0,AU,1.660448e+09,55.0,-24.8667,113.6333,70.52,1.66,Hospitality Carnarvon
166,183,havelock,0.0,US,1.660448e+09,78.0,34.8791,-76.9013,75.94,5.75,Sherwood Motel
186,203,sorrento,0.0,IT,1.660448e+09,90.0,40.6260,14.3744,72.97,0.00,Hilton Sorrento Palace
296,327,mayo,0.0,US,1.660448e+09,82.0,38.8876,-76.5119,72.99,0.00,Resorts Resource Group International
444,493,fairbanks,0.0,US,1.660448e+09,63.0,64.8378,-147.7164,73.60,0.00,Westmark Fairbanks Hotel & Conference Center
518,571,vernon,0.0,CA,1.660448e+09,42.0,50.2581,-119.2691,74.01,1.14,Prestige Vernon Lodge and Conference Centre
550,604,andros,0.0,GR,1.660448e+09,54.0,37.8333,24.9333,73.96,4.00,Micra Anglia Hotel - Andros
561,616,rockhampton,0.0,AU,1.660448e+09,41.0,-23.3833,150.5000,76.89,4.61,Mercure Rockhampton


In [67]:
# Assign the marker layer to a variable
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))